In [ ]:
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import re

In [ ]:
df = pd.read_csv('../data/raw/phrasebank_raw.csv')
df

In [ ]:
df['clean_text'] = df['sentence']
df = df[['sentence', 'clean_text', 'sentiment', 'word_count', 'char_count']]
df['clean_text'] = df['clean_text'].str.lower()
df

In [ ]:
df['clean_text'] = df['clean_text'].str.strip()
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub(r'\W+[\.,]?\W*', ' ', x))
df['clean_text'] = df['clean_text'].apply(lambda x: re.sub(r'\d+[\.,]?\d*', '<NUM>', x))


In [ ]:
import nltk
from nltk.corpus import stopwords
nltk.download('punkt_tab')
nltk.download('stopwords')
nltk.download('wordnet')

df['clean_text'] = df['clean_text'].apply(nltk.word_tokenize)

stop_words = set(stopwords.words('english'))
df['clean_text'] = df['clean_text'].apply(lambda rem:[word for word in rem if word not in stop_words])


In [ ]:
from nltk.stem import WordNetLemmatizer
wnl = WordNetLemmatizer()
df['clean_text'] = df['clean_text'].apply(lambda tok:[wnl.lemmatize(word) for word in tok])
df['clean_text'] = df['clean_text'].apply(lambda strin:" ".join(strin) )
clean_df = df[['clean_text','sentiment']]
clean_df.to_csv("../data/processed/clean_df.csv", index=False)

In [ ]:
from sklearn.model_selection import train_test_split

X = df['clean_text']
y = df['sentiment']

X_train_val, X_test, y_train_val, y_test = train_test_split(X, y, test_size=0.3, stratify=y, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.5, random_state=42, stratify=y_train_val)

In [ ]:
train_csv = pd.concat([X_train,y_train],axis=1)
train_csv.to_csv("../data/processed/train.csv", index=False)
valid_csv = pd.concat([X_val,y_val],axis=1)
valid_csv.to_csv("../data/processed/valid.csv", index=False)
test_csv = pd.concat([X_test,y_test],axis=1)
test_csv.to_csv("../data/processed/test.csv", index=False)